# N.I.M.S.
Notebook Interface for Member Scraping

In [3]:
from MEATY_backend import *
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from bs4 import BeautifulSoup as BS
import time
import re
import pandas as pd
import numpy as np

In [4]:
def process_profile_html(profile_block):
    member_data = {}
    name, fb_id = process_name_fbid(profile_block)
    member_data['name'] = name
    member_data['fb_id'] = fb_id
    ref, date = process_member_add_data(profile_block)
    member_data['ref'] = ref
    member_data['date'] = date
    affil = profile_block.find(attrs={'class': '_17tq'}).text
    member_data['affil'] = affil
    return member_data

In [5]:
def process_name_fbid(block):
    name_id_block = block.find_all('a')
    for elem in name_id_block:
        name = elem.text
        if name != '':
            fb_id = str(name_id_block).split('id=')[1].split('&amp;')[0]
            break
    return name, int(fb_id)

In [6]:
def process_member_add_data(profile_block):
    today = datetime.date.today()
    try:
        added_by = profile_block.find(string=re.compile('Added by'))[9:]
    except:
        added_by = None
        try:
            date_added = profile_block.find(class_='timestampContent').text
        except:
            date_added = profile_block.find(class_='fsm fwn fcg').text
            if 'Created group on December 5, 2016':
                # datetime.date(2016, 12, 5)
                date_added = ' '.join(date_added.split(' ')[-3:])
                added_by = 'GROUP CREATOR'
            else:
                date_added = None
                added_by = None
                print('<< PROBLEMATIC >>\n', profile_block.prettify())
            return added_by, date_added
        if 'Joined about' in date_added:
            timedelta = int(date_added.split(' ')[2])
            if 'months' in date_added:
                date_added = today - time_ago(months=+timedelta)
            elif 'years' in date_added:
                date_added = today - time_ago(years=+timedelta)
            return None, date_added
        elif '201' not in date_added:
            date_added = date_added + ", 2017"
        date_added = datetime.datetime.strptime(date_added, '%B %d, %Y').date()
        return added_by, date_added
    if '2016' in added_by  \
            or '2017' in added_by  \
            or '2015' in added_by  \
            or '2014' in added_by:
        added_by, date_added = added_by.split(' on ')
        date_added = datetime.datetime.strptime(date_added, '%B %d, %Y').date()
    elif ' Today' in added_by:
        added_by = added_by.rstrip(' Today')
        date_added = today
    elif ' Yesterday' in added_by:
        added_by = added_by.rstrip(' Yesterday')
        date_added = today - time_ago(days=+1)
    else:
        print('added_by/date_added: Cant parse', added_by)
        added_by = '~~' + added_by
        date_added = added_by
    return added_by, date_added

In [9]:
def html_to_df(soup=None, html_path='../../cbsm.html', save_csv=True):
    if soup is None:
        with open(html_path) as f:
            soup = BS(f, 'lxml')
    ppl = soup.find_all(attrs={'class': "_6a _5u5j _6b"})
    member_data = []
    for memester in ppl:
        member_data.append(process_profile_html(memester))
    member_df = pd.DataFrame(member_data).set_index('fb_id')
    if save_csv:
        member_df.to_csv('columbia-member_data.csv')
    return member_df

In [8]:
#   NEED TO FIX for case where no old df
def update_members(fb_email_or_phone=None, fb_pass=None, old_df=None, save_csv=True):
    browser = login_to_fb(fb_email_or_phone, fb_pass)
    soup = BS(browser.page_source, 'lxml')
    ppl = soup.find_all(attrs={'data-name': "GroupProfileGridItem"})
    last_on_page = ppl[-1]
    name, fb_id = process_name_fbid(last_on_page)
    new_people = old_df[old_df.fb_id == fb_id].shape[0] == 0
    i = 1
    time.sleep(1.5)
    while new_people:
        print(i)
        i += 1
        try:
            load_more = browser.find_element_by_class_name(
                'uiMorePagerPrimary')
        except:
            time.sleep(1)
            load_more.click()
        load_more.click()
        soup = BS(browser.page_source, 'lxml')
        ppl = soup.find_all(attrs={'data-name': "GroupProfileGridItem"})
        last_on_page = ppl[-1]
        name, fb_id = process_name_fbid(last_on_page)
        new_people = old_df[old_df.fb_id == fb_id].shape[0] == 0
        time.sleep(1.5)
    new_ppl_df = html_to_df(soup, save_csv=False)
    new_ppl_ids = np.array(list(set(new_ppl_df.index) - set(old_df.fb_id)))
    new_ppl_df = new_ppl_df.loc[new_ppl_ids].reset_index()
    new_df = pd.concat([new_ppl_df, old_df])
    if save_csv:
        fn_str = 'member_data_' + datetime.datetime.today().strftime('%m-%d-%Y_%H-%M') + '.csv'
        new_df.set_index('fb_id').to_csv(fn_str)
    return new_df

In [5]:
def load_more_members(browser, group_name='cornell'):
    i = 0
    print("+++++++   FRESH START   +++++++")
    while True:
        print(i)
        i += 1
        try:
            load_more = browser.find_element_by_class_name(
                'uiMorePagerPrimary')
            load_more.click()
        except:
            time.sleep(1)
            try:
                load_more = browser.find_element_by_class_name(
                    'uiMorePagerPrimary')
                load_more.click()
            except:
                time.sleep(4)
                try:
                    load_more = browser.find_element_by_class_name(
                        'uiMorePagerPrimary')
                    load_more.click()
                except:
                    print('-------   DONE   -------')
                    alt_html = browser.execute_script(
                        "return document.body.innerHTML")
                    soup_alt = BS(alt_html, 'lxml')
                    indf = html_to_df(soup_alt, save_csv=False)
                    indf.to_csv(group_name + '-html_alt.csv')
                    html = browser.page_source
                    soup = BS(html, 'lxml')
                    df = html_to_df(soup, save_csv=False)
                    df.to_csv(group_name + '-html.csv')
                    return browser, df, indf

In [10]:
browser = login_to_fb('https://www.facebook.com/groups/makecornellmemeagain/members/?order=date')
b, df, indf = load_more_members(browser)

+++++++   FRESH START   +++++++
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
-------   DONE   -------
+++++++   FRESH START   +++++++
0
-------   DONE   -------


(21157, 4)